# Build Convolutional Neural Network

<font color='steelblue'>

<span style="font-family:verdana; font-size:1.6em;">
    <strong>MNIST Fashion Dataset</strong><br><br>
    From the Keras datasets, import the MNIST Fashion data.<br>
    There are images of 10 different fashion items which have labels associated
    with each image.<br>
</span>

</font>
<font color = 'tomato'>
    <h2>Implementation requirements defined in red</h2>
</font>

In [ ]:
%config IPCompleter.greedy = True

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

In [ ]:
# make sure tensorflow is properly installed
tf.__version__, tf.keras.__version__

## Locate the dataset

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = \
                                fashion_mnist.load_data()

In [ ]:
# make a list of the class names (index presents the class)
class_names = ['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

## Explore Data

In [ ]:
print("Size: train images {}, train labels {}".format(train_images.shape, 
                                                      train_labels.shape))

# save the number of items in training dataset
train_rows = train_images.shape[0]

In [ ]:
print("Size: test images {}, test labels {}".format(test_images.shape, 
                                                      test_labels.shape))

# save the number of items in test dataset
test_rows = test_images.shape[0]

In [ ]:
# look at first 10 labels in training set
train_labels[:10]

In [ ]:
plt.figure
plt.imshow(train_images[1], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

<font color='tomato'>

<span style="font-family:verdana; font-size:1.5em;">
    <b>Implement the following:</b><br>
    <ol>
        <li>Normalize the images</li>
        <li>Plot first 25 images</li>
        <li>Reshape images to 28x28x1</li>
    </ol>
</span>
</font>

In [ ]:
# Normalize the values to be between 0 and 1; min-max normalization

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
# define the image shape (horizontal pixels x vertical pixels x grey scale)
iShape = (28,28, 1)

In [ ]:
# The data needs to be reshaped.
# The first convolution expects a single tensor containing everything, 
# so instead of 60,000 28x28x1 items in an array (60000, 28, 28), 
# it wants a single 4D array/tensor that is 60000x28x28x1, otherwise you will get an error.

train_images = train_images.reshape(train_rows, 28, 28, 1)

# reshape test images into a single tensor
test_images = test_images.reshape(test_rows, 28, 28, 1)

<font color='tomato'>
<h2>Convolutional Neural Network:</h2><br>
<span style="font-family:verdana; font-size:1.5em;">
    <b>Implement the following:</b><br>
    <ol>
        <li>Create a keras sequential model (define appropriate layers</li>
        <li>Compile the model</li>
        <li>Print the summary for the model</li>
    </ol>
</span>
</font>

In [ ]:
# Create Convolution Neural Network - adding layers

model = keras.models.Sequential()
model.add(layers.Conv2D(32, activation = 'relu', input_shape = iShape, kernel_size=(3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))   # (3,3) - filter size, 64 number of filters
model.add(layers.MaxPooling2D(pool_size=(2, 2)))          # (2,2) - pooling size

In [ ]:
# To avoid over fitting on training dataset, randomly drop neurons and their connections
# remove 25% of them
model.add(layers.Dropout(rate = 0.25))

In [ ]:
# Convert the previous layer into 1 dimensional array (flatten it)
model.add(layers.Flatten())

In [ ]:
# Once we’ve flattened the data into a 1D array, add a dense hidden layer, which is normal 
# to a traditional neural network. Next, add another dropout layer before 
# adding a final dense layer which classifies the data
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))   # helps classify into classes

In [ ]:
# Compile the model with chosen parameters
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Train Convolutional Neural Network (CNN
<h3><b>Used 10 epochs and got about 99.08% accuracy on training and the validation set</b></h3>

<font color='tomato'>
<h2>Train Convolutional Neural Network:</h2><br>
<span style="font-family:verdana; font-size:1.5em;">
    <b>Implement the following:</b><br>
    <ol>
        <li>Train the model defined</li>
        <li>Make sure to time it</li>
        <li>Display the accuracy, loss information</li>
    </ol>
</span>
</font>

In [ ]:
# To measure time required to train
import timeit, time
start = timeit.default_timer()

In [ ]:
# Train the model and include a validation set (composed of 10% of the dataset)
# Capturing the returned history enables you to plot the change in 
# error/loss and accuracy over time
batches = 128
epochs = 6
history = model.fit(train_images, train_labels, validation_split = 0.2, 
                    batch_size = batches, epochs = epochs)

In [ ]:
stop = timeit.default_timer()
execution_time = stop - start
exectime = time.strftime("%M:%S", time.gmtime(execution_time)) 
print("To train it took: {} mins".format(exectime))

In [ ]:
metrics_names = model.metrics_names

In [ ]:
# Function to plot the accuracy and loss
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.title('Training and validation')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
for name in metrics_names:
    plot_graphs(history, name)

<font color='tomato'>
<h2>Evaluate Convolutional Neural Network Model:</h2><br>
<span style="font-family:verdana; font-size:1.5em;">
    <b>Implement the following:</b><br>
    <ol>
        <li>Evaluate the model with test images</li>
        <li>Print accuracy</li>
    </ol>
</span>
</font>

In [ ]:
# Use the test images to evaluate the model on a set of unseen images

test_loss, test_acc = model.evaluate(test_images, test_labels)


In [ ]:
print("Test accuracy: ", test_acc)

## Model Evaluation

In [ ]:
# predict_classes() returns a class prediction (i.e., the class with the highest probability)
# predict() returns probabilities
predictions = model.predict(test_images)

In [ ]:
# preview some of the test_images labels
test_labels[0:10]

In [ ]:
# view the prediction for a selected test_image
predictions[9]
# predictions[9].round(2) returns probabilities to 2 decimal places

In [ ]:
# sort the probabilities (from least likely class to the most likely class)
np.argsort(predictions[9])

In [ ]:
# you can reverse the order to view the class probabilities from best to worst
np.argsort(predictions[9])[::-1]

In [ ]:
# display the class number with the highest probability
    # similar to using moldel.predict_classes()
np.argmax(predictions[9])

In [ ]:
# display the class name with the highest probability; the class name 
# with the index position of the prediction
class_selection = np.argmax(predictions[9])
class_names[class_selection]

## Make prediction on single new image

In [ ]:
# Make a prediction on a single (new) image
# Grab an image. Here we'll select one from our test_images.

img = test_images[0]
img.shape

In [ ]:
# tf.keras models are optimized to make predictions on a batch, or collection, of 
# examples at once. So, even though we're using a single image, we need to add 
# it to a batch of one.

# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,axis=0))
img.shape

In [ ]:
# use the trained model to predict the class that the image belongs to
single_prediction = model.predict(img)
print(single_prediction[0])

In [ ]:
# display the prediction
classification = np.argmax(single_prediction[0])
class_name = class_names[classification]
                           
print("The predicted class label is {}: {}".format(classification, class_name))

In [ ]:
# display the correct label
print("The true class label is {}: {}".format(test_labels[0], class_names[test_labels[0]]))

<span style="font-family:Arial; font-size:1.2em;">
<font color='tomato'>
    <h2>Practice</h2>
    <h3>Try out different parameters and see how model accuracy changes</h3>
    <ol>
        <li>Play with different epoch values (10, 20, ...)</li>
        <li>Add more Conv2D and Pooling layers</li>
        <li>Change number of neuron in each dense layer</li>
        <li>Change the batch size and see what happens</li>    
    </ol>
</font>
</span>